In [29]:
import feedparser
from openai import OpenAI 
import datetime
import json

client = OpenAI(api_key = OPEN_API_KEY)

# URL of the RSS feed
rss_url = "https://www.smh.com.au/rss/business.xml"

def fetch_rss_entries(rss_url):
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    # print(feed)
    entries = feed.entries
    return entries

def get_title_summary(entries):
    entries_list = []
    for entry in entries:
        dict = {}
        dict['title'] = entry.title 
        dict['summary'] = entry.summary
        entries_list.append(dict)
    return(entries_list)

def summarize_news_with_chatgpt(articles):        
    prompt = (
        """
        Summarize the Top 10 articles from the Sydney Morning Herald RSS feed, focusing on key investment insights. 
    Highlight any important market trends, stock movements, or economic forecasts that could impact personal investment decisions. 
    For each article, provide a brief analysis of how the information may affect different asset classes (stocks, bonds, real estate, etc.), and offer actionable takeaways for personal investors.
    """
        + str(articles)
    )
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # Use GPT-4o-mini/gpt-3.5-turbo for the latest model
            messages=[
                {"role": "system", "content": "You are a financial expert summarizing daily financial news for the Australian and APAC (Asia Pacific) stock markets."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=1500,
            temperature=0.5,
        )
        return response.choices[0].message.content        
        # return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

if __name__ == "__main__":
    # Fetch the RSS feed entries
    entries = fetch_rss_entries(rss_url)

    # print(entries)


    # # Summarize the news using ChatGPT
    if entries:
        title_summaries = get_title_summary(entries)
        summary = summarize_news_with_chatgpt(title_summaries)
        print("Summary of the day's financial news:")
        print(summary)
    else:
        print("No news entries found.")


Summary of the day's financial news:
### 1. **Auric pinpoints future targets with new WA gold hits**
**Summary:** New drill results at Auric Mining’s Spargoville play in WA, including 4m at 2.79g/t gold and 22m at 0.246g/t, hint at potential treasures lurking below.
**Analysis:** Positive drill results suggest potential for significant gold deposits, which could boost Auric Mining's stock. 
**Takeaway:** Investors may consider adding Auric Mining to their portfolios, anticipating future growth in stock value.

### 2. **Inside the race to crack elusive WA nickel-copper code**
**Summary:** Companies are striving to crack the exploration code within WA’s Albany-Fraser Orogen to reveal another Nova-Bollinger nickel-copper find.
**Analysis:** Success in this endeavor could lead to substantial stock gains for companies involved in exploration.
**Takeaway:** Keep an eye on companies engaged in WA’s Albany-Fraser Orogen; successful discoveries could present lucrative investment opportunities.


In [30]:
#Build JSON object
datetoday = datetime.datetime.today().strftime('%Y%m%d')
dailynews_json = {"date": datetoday, "summary": summary}

In [31]:
#Write to JSON to local
with open(str(datetoday+"_SMH.json"), "w") as write_file:
    json.dump(dailynews_json, write_file)